In [1]:
!pip install rasterio geojson rasterstats owslib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.1/240.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 89.5 MB/s eta 0:00:00


In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd
from owslib.wcs import WebCoverageService
import requests
import geojson
import rasterio
import geopandas as gpd
from rasterio.plot import show
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from rasterstats import zonal_stats
from io import BytesIO
from shapely.geometry import box
from matplotlib.colors import ListedColormap, BoundaryNorm


## Importando camada vetorial

In [3]:
# Parâmetros para conectar com a camada vetorial
mun_url = "https://info.dengue.mat.br/geoserver/wfs"

params_AML = dict(
    service="WFS",
    version="2.0.0",
    request="GetFeature",
    typeName="AP_subdistritos_CD2022", # usando camada de subdistritos, pois municípios na Amazônia são gigantes
    outputFormat="json",
)

# Fazendo o request
r_AML = requests.get(mun_url, params=params_AML)

# Baixando e carregando a camada
municipios = gpd.GeoDataFrame.from_features(geojson.loads(r_AML.content), crs="EPSG:4674")


In [4]:
municipios.head()

,geometry,id,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_SUBDIST,NM_SUBDIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,SIGLA
0,"POLYGON ((-52.00182 0.93616, -52.00186 0.93637...",1,1,Norte,16,Amapá,1600055,Serra do Navio,160005505,Serra do Navio,16000550500,None,1602,Oiapoque - Porto Grande,160004,Porto Grande,None,None,AP
1,"POLYGON ((-52.01806 0.88724, -52.01771 0.8863,...",2,1,Norte,16,Amapá,1600055,Serra do Navio,160005510,Cachaço,16000551000,None,1602,Oiapoque - Porto Grande,160004,Porto Grande,None,None,AP
2,"POLYGON ((-50.45011 2.10924, -50.44715 2.10917...",3,1,Norte,16,Amapá,1600105,Amapá,160010505,Amapá,16001050500,None,1602,Oiapoque - Porto Grande,160003,Oiapoque,None,None,AP
3,"POLYGON ((-50.5088 2.18518, -50.5069 2.18443, ...",4,1,Norte,16,Amapá,1600105,Amapá,160010505,Amapá,16001050500,None,1602,Oiapoque - Porto Grande,160003,Oiapoque,None,None,AP
4,"POLYGON ((-50.77464 1.83875, -50.7747 1.8383, ...",5,1,Norte,16,Amapá,1600105,Amapá,160010505,Amapá,16001050500,None,1602,Oiapoque - Porto Grande,160003,Oiapoque,None,None,AP


In [5]:
# Filtrar para o município de estudo
municipio_estudo = "Oiapoque"

municipio_bbox = municipios[municipios['NM_DIST'].str.contains(municipio_estudo, case=False)]

# Obter os limites (bounding box) do Oiapoque/AP
xmin, ymin, xmax, ymax = municipio_bbox.total_bounds
xmin, ymin, xmax, ymax


(np.float64(-51.855426),
 np.float64(3.17639),
 np.float64(-51.356847),
 np.float64(4.394141))

In [6]:
municipio_bbox.total_bounds

array([-51.855426,   3.17639 , -51.356847,   4.394141])

## Importando rasters de 2008 e 2022

### 1 - Descobrindo o tamanho em pixels do raster

In [7]:
import requests
import xml.etree.ElementTree as ET

# Parâmetros para o WCS e a cobertura
wcs_url = "https://info.dengue.mat.br/geoserver/wcs"
wcs_version = "1.0.0"
coverage_id = "brasil_uso_cob:mapbiomas_brasil_coverage_2008"

# Monta a URL para o DescribeCoverage (sem espaços, com os parâmetros necessários)
describe_url = (f"{wcs_url}?service=WCS&version={wcs_version}"
                f"&request=DescribeCoverage&coverage={coverage_id}")

r_describe = requests.get(describe_url)
if r_describe.status_code != 200:
    raise Exception("Erro no DescribeCoverage: " + str(r_describe.status_code))

# Parseia o XML da resposta
root = ET.fromstring(r_describe.content)
# Definir os namespaces – pode ser necessário ajustar se o XML usar outros valores
ns = {
    "wcs": "http://www.opengis.net/wcs",
    "gml": "http://www.opengis.net/gml"
}

# Procura o elemento GridEnvelope na resposta (normalmente dentro de CoverageDescription)
grid_env = root.find(".//gml:GridEnvelope", ns)
if grid_env is None:
    raise Exception("Nenhum elemento <gml:GridEnvelope> encontrado no DescribeCoverage.")

low_elem = grid_env.find("gml:low", ns)
high_elem = grid_env.find("gml:high", ns)
if low_elem is None or high_elem is None:
    raise Exception("Elementos <gml:low> ou <gml:high> não foram encontrados no GridEnvelope.")

# Os valores geralmente vêm como uma string com dois números separados por espaço
low_vals = list(map(int, low_elem.text.split()))
high_vals = list(map(int, high_elem.text.split()))
native_width = high_vals[0] - low_vals[0] + 1
native_height = high_vals[1] - low_vals[1] + 1
print("Dimensões nativas extraídas (width x height):", native_width, "x", native_height)


Dimensões nativas extraídas (width x height): 155241 x 158828


### 2 - Calculando o tamanho em pixels do recorte que faremos (para manter a mesma resolucão)

In [8]:
# Metadados do raster original (exemplo)
orig_bbox = (-73.98318216, -16.66197917, -43.39929216, 5.26958083)  # (xmin, ymin, xmax, ymax)
native_width = 155241   # largura original em pixels
native_height = 158828  # altura original em pixels

# Calcular resolução em x e y
res_x = (orig_bbox[2] - orig_bbox[0]) / native_width
res_y = (orig_bbox[3] - orig_bbox[1]) / native_height
print("Resolução em x:", res_x)
print("Resolução em y:", res_y)

# Definir a bbox do recorte (por exemplo, uma região de interesse)
crop_bbox = (float(xmin), float(ymin), float(xmax), float(ymax))  # convertendo de np.float para float simples
# crop_bbox = (-52.7506, 3.6626, -51.7776, 4.0039)

# Calcular dimensões do recorte em pixels
crop_width_pixels = int(round((crop_bbox[2] - crop_bbox[0]) / res_x))
crop_height_pixels = int(round((crop_bbox[3] - crop_bbox[1]) / res_y))

print("Dimensões nativas do recorte (pixels): {} x {}".format(crop_width_pixels, crop_height_pixels))


Resolução em x: 0.00019700910197692617
Resolução em y: 0.000138083713199184
Dimensões nativas do recorte (pixels): 2531 x 8819


In [9]:
crop_bbox

(-51.855426, 3.17639, -51.356847, 4.394141)

In [10]:
coverage_id = "amazonia_terraclass:terraclass_AMZ.2008.M"
crs = "EPSG:4674"
output_format = "image/geotiff"

wcs = WebCoverageService(wcs_url, version=wcs_version, timeout=None)

# Executa a requisição para obter a cobertura
response_2008 = wcs.getCoverage(
    identifier=coverage_id,
    format=output_format,
    crs=crs,
    bbox=crop_bbox,#(-51.855426, 3.17639, -51.356847, 4.394141), # não consigo usar o objeto crop_bbox, por isso o hardcoding
    # bbox=(-52.7506, 3.6626, -51.7776, 4.0039), # zoom na área urbana do Oiapoque
    width=crop_width_pixels,
    height=crop_height_pixels,
    timeout=None
)

output_file = "terraclass_2008.tif"
# output_file = "/content/drive/MyDrive/Rasters/terraclass_2008.tif"
with open(output_file, "wb") as f:
    f.write(response_2008.read())


In [11]:
# Lista de anos com Terraclass disponível
years = [2008, 2010, 2012, 2014, 2018, 2020, 2022]
crs = "EPSG:4674"

for year in years:
  coverage_id = f"amazonia_terraclass:terraclass_AMZ.{year}.M"
  output_file = f"terraclass_{year}.tif"

  print(f"requisitando cobertura {coverage_id}")

  response = wcs.getCoverage(
      identifier=coverage_id,
      format=output_format,
      crs=crs,
      bbox=crop_bbox,
      width=crop_width_pixels,
      height=crop_height_pixels,
      timeout=None
      )

  with open(output_file, "wb") as f:
    f.write(response.read())

  print(f"{output_file} salvo com sucesso")


requisitando cobertura amazonia_terraclass:terraclass_AMZ.2008.M
terraclass_2008.tif salvo com sucesso
requisitando cobertura amazonia_terraclass:terraclass_AMZ.2010.M
terraclass_2010.tif salvo com sucesso
requisitando cobertura amazonia_terraclass:terraclass_AMZ.2012.M
terraclass_2012.tif salvo com sucesso
requisitando cobertura amazonia_terraclass:terraclass_AMZ.2014.M
terraclass_2014.tif salvo com sucesso
requisitando cobertura amazonia_terraclass:terraclass_AMZ.2018.M
terraclass_2018.tif salvo com sucesso
requisitando cobertura amazonia_terraclass:terraclass_AMZ.2020.M
terraclass_2020.tif salvo com sucesso
requisitando cobertura amazonia_terraclass:terraclass_AMZ.2022.M
terraclass_2022.tif salvo com sucesso


In [20]:
src_2008 = rasterio.open("terraclass_2008.tif")
src_2010 = rasterio.open("terraclass_2010.tif")
src_2012 = rasterio.open("terraclass_2012.tif")
src_2014 = rasterio.open("terraclass_2014.tif")
src_2018 = rasterio.open("terraclass_2018.tif")
src_2020 = rasterio.open("terraclass_2020.tif")
src_2022 = rasterio.open("terraclass_2022.tif")


In [21]:
years = [2008, 2010, 2012, 2014, 2018, 2020, 2022]

data = {}
profiles = {}
nodata_vals = {}
transforms = {}

for year in years:
  path = f"terraclass_{year}.tif"
  print(f"Lendo {path}…")
  with rasterio.open(path) as src:
    # lê banda 1 como array NumPy
    data[year] = src.read(1)
    # metadados (dimensão, CRS, dtype, etc.)
    profiles[year] = src.profile
    # valor de nodata
    nodata_vals[year] = src.nodata
    # transform affine
    transforms[year] = src.transform


Lendo terraclass_2008.tif…
Lendo terraclass_2010.tif…
Lendo terraclass_2012.tif…
Lendo terraclass_2014.tif…
Lendo terraclass_2018.tif…
Lendo terraclass_2020.tif…
Lendo terraclass_2022.tif…


In [22]:
data_2008 = data[2008]
data_2010 = data[2010]
data_2012 = data[2012]
data_2014 = data[2008]
data_2018 = data[2018]
data_2020 = data[2020]
data_2022 = data[2022]


In [23]:
profile_2008 = profiles[2008]
profile_2010 = profiles[2010]
profile_2012 = profiles[2012]
profile_2014 = profiles[2014]
profile_2018 = profiles[2018]
profile_2020 = profiles[2020]
profile_2022 = profiles[2022]


In [24]:
nodata_2008 = nodata_vals[2008]
nodata_2010 = nodata_vals[2010]
nodata_2012 = nodata_vals[2012]
nodata_2014 = nodata_vals[2014]
nodata_2018 = nodata_vals[2018]
nodata_2020 = nodata_vals[2020]
nodata_2022 = nodata_vals[2022]


In [25]:
transform_2008 = transforms[2008]
transform_2010 = transforms[2010]
transform_2012 = transforms[2012]
transform_2014 = transforms[2014]
transform_2018 = transforms[2018]
transform_2020 = transforms[2020]
transform_2022 = transforms[2022]


In [26]:
# Se os rasters tiverem valor NoData, mascaramos esses pixels para que apareçam em branco
if nodata_2008 is not None:
    data_2008 = np.ma.masked_equal(data_2008, nodata_2008)
if nodata_2010 is not None:
    data_2010 = np.ma.masked_equal(data_2010, nodata_2010)
if nodata_2012 is not None:
    data_2012 = np.ma.masked_equal(data_2008, nodata_2008)
if nodata_2014 is not None:
    data_2014 = np.ma.masked_equal(data_2008, nodata_2008)
if nodata_2018 is not None:
    data_2018 = np.ma.masked_equal(data_2008, nodata_2008)
if nodata_2020 is not None:
    data_2020 = np.ma.masked_equal(data_2008, nodata_2008)
if nodata_2022 is not None:
    data_2022 = np.ma.masked_equal(data_2008, nodata_2008)


In [27]:
extent = [src_2008.bounds.left, src_2008.bounds.right, src_2008.bounds.bottom, src_2008.bounds.top]
extent


[-51.855426, -51.356847, 3.17639, 4.394141]

## Extraindo estatísticas zonais

In [34]:
municipio_estudo = "Oiapoque"

mun_estudo = municipios[municipios['NM_DIST'].str.contains(municipio_estudo, case=False)]


In [38]:
years = [2008, 2010, 2012, 2014, 2018, 2020, 2022]
zs = {}

for year in years:
  zs[year] = zonal_stats(mun_estudo, data[year], affine=transforms[year], categorical=True, nodata=nodata_vals[year])



In [43]:
zs[2008]

[{1: 6063380,
  2: 144616,
  10: 160058,
  11: 44881,
  16: 386,
  17: 12756,
  20: 1649,
  22: 16642,
  23: 333381,
  25: 56245,
  50: 3624171}]

In [51]:
# extraímos o único elemento de cada lista e montamos um dict “ano = {classe: contagem, …}”
flat = {year: zs[year][0] for year in years}

# criamos o DataFrame, transpondo para que cada linha seja um ano
df = pd.DataFrame.from_dict(flat, orient='index').fillna(0)

# converte o índice (ano) em coluna
df.index.name = 'year'
df.reset_index(inplace=True)

print(df)


   year        1       2      10      11     16     17       20     22  \
0  2008  6063380  144616  160058   44881  386.0  12756   1649.0  16642   
1  2010  6052690  136562   80012   33941  209.0  15768    919.0   4741   
2  2012  6043844  198583   38925   23522  101.0  16692   2288.0   5002   
3  2014  6033960  195768   93271   33917  153.0  17799  11520.0   2218   
4  2018  6019709  184753  117320  129894    0.0  12734      0.0   6620   
5  2020  6001175  188795   87016  169417    0.0  23850    500.0   1544   
6  2022  5996889  195882   98057  152124    0.0  26388    495.0   4154   

       23      25         50         51  
0  333381   56245  3624171.0        0.0  
1  333381  175704  3624238.0        0.0  
2  333381  171595  3624232.0        0.0  
3  333381  112091  3624087.0        0.0  
4  333427   40703        0.0  3613005.0  
5  333381   40712        0.0  3611775.0  
6  333381   40712        0.0  3610083.0  


In [52]:
# Dicionário de mapeamento
palette = {
    1:  "vegetacao_natural_florestal_primaria",
    2:  "vegetacao_natural_florestal_secundaria",
    9:  "silvicultura",
    10: "pastagem_arbustiva_arborea",
    11: "pastagem_herbacea",
    12: "cultura_agricola_perene",
    13: "cultura_agricola_semiperene",
    14: "cultura_agricola_temporaria_1_ciclo",
    15: "cultura_agricola_temporaria_mais_de_1_ciclo",
    16: "mineracao",
    17: "urbanizada",
    20: "outros_usos",
    22: "desflorestamento_no_ano",
    23: "corpo_dagua",
    25: "não_observado",
    50: "não_floresta",
    51: "natural_nao_florestal",
    52: "cultura_agricola_temporaria",
}

# Monta um dicionário de renomeação só com as colunas que existem em df
col_mapping = {}
for k, v in palette.items():
    if k in df.columns:
        col_mapping[k] = v
    elif str(k) in df.columns:
        col_mapping[str(k)] = v

# Aplica o renomeamento
df_final = df.rename(columns=col_mapping)

print(df_final.columns)


Index(['year', 'vegetacao_natural_florestal_primaria',
       'vegetacao_natural_florestal_secundaria', 'pastagem_arbustiva_arborea',
       'pastagem_herbacea', 'mineracao', 'urbanizada', 'outros_usos',
       'desflorestamento_no_ano', 'corpo_dagua', 'não_observado',
       'não_floresta', 'natural_nao_florestal'],
      dtype='object')


In [53]:
print(df_final)


   year  vegetacao_natural_florestal_primaria  \
0  2008                               6063380   
1  2010                               6052690   
2  2012                               6043844   
3  2014                               6033960   
4  2018                               6019709   
5  2020                               6001175   
6  2022                               5996889   

   vegetacao_natural_florestal_secundaria  pastagem_arbustiva_arborea  \
0                                  144616                      160058   
1                                  136562                       80012   
2                                  198583                       38925   
3                                  195768                       93271   
4                                  184753                      117320   
5                                  188795                       87016   
6                                  195882                       98057   

   pastagem_herbacea  